In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
# other modules
import numpy as np
from tqdm import trange
import pickle
# internal imports
from utils.attacks.sparse_rs import RSAttack
from utils.models import *
from utils.helpers import *

import torch
import argparse
import json
# PATHING
import os
import sys
root = os.path.abspath(os.curdir)
sys.path.append(root)

from utils.attack import attack
# local imports
# from utils.adv_trainer import adv_trainer

In [2]:
# settings for run
device = 'cuda:3'
net_path = '/home/mbeliaev/home/code/robust-l0/new_trained/test_mnist/cnn_large_k0_p12_seed99/'
# net_path = '/home/mbeliaev/home/code/robust-l0/new_trained/test_cifar/cnn_large_k0_p12_seed99/'
# this contains f_results.p, net.pth, results.p, setup.json
config = json.load(open(net_path+'setup.json','rb'))
print('Loading configuration:')
for key in config.keys():
    print('\t', key,': ', config[key])

# load model
Data = prep_data(root, config['bs'], config['dataset'])
net = Net(config['arch'], config['k'], config['embedding'], Data['x_test'][0].shape)
net = net.to(device)
net.load_state_dict(torch.load(net_path+'net.pth', map_location=device))
net.eval()

Loading configuration:
	 arch :  cnn_large
	 no_adv :  True
	 device :  cuda:3
	 dataset :  MNIST
	 exp :  test_mnist
	 out_dir :  new_trained
	 k :  0
	 perturb :  12
	 seed :  99
	 bs :  128
	 lr :  0.1
	 momentum :  0.9
	 decay :  0.5
	 embedding :  256
	 iters :  5
	 queries :  300
	 epochs :  10


Net(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(64, 128, kern

In [3]:
result_str = pickle.load(open(net_path+'results.p', 'rb'))
for string in result_str: print(string)

Running iter 0
Accuracy on testset: 97.380
Accuracy on trainset: 97.932
Running iter 1
Accuracy on testset: 98.950
Accuracy on trainset: 99.998
Running iter 2
Accuracy on testset: 98.970
Accuracy on trainset: 99.998
Running iter 3
Accuracy on testset: 98.940
Accuracy on trainset: 99.998
Running iter 4
Accuracy on testset: 98.940
Accuracy on trainset: 100.000


In [4]:
# lets quickly confirm this result
evaluate(net, Data['x_test'], Data['y_test'], device)

98.94

In [5]:
r_acc, x_adv, y_adv, i_adv = attack(net= net,
                            k = 12,
                            x = Data['x_test'],
                            y = Data['y_test'],
                            n_queries=100,
                            n_restarts=1,
                            device=device,
                            log_path=net_path+'rs_log.txt'
                            )

/home/mbeliaev/home/code/robust-l0/utils/attacks/sparse_rs.py:952: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370172916/work/torch/csrc/utils/python_arg_parser.cpp:882.)
  ind_to_fool = acc.nonzero().squeeze()


In [9]:
len(x_adv[0])

74